In [3]:
# import torch
import numpy as np
import data_pipes

df_path = "processed_aapl_data.csv"
data = data_pipes.process_df(df_path)

In [4]:
from torch import nn

class georgia_0(nn.Module):
    def __init__(self, win_past=20, features=19, dropout_rate=0.4):
        super().__init__()
        self.flat0 = nn.Flatten()

        self.hidden0 = nn.Linear(features*win_past, 128)
        self.act0 = nn.ReLU()
        self.dropout0 = nn.Dropout(dropout_rate)
        
        self.hidden1 = nn.Linear(128, 128)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.hidden2 = nn.Linear(128, 128)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.output = nn.Linear(128, 1)
    def forward(self, x):
        x = self.flat0(x)
        x = self.dropout0(self.act0(self.hidden0(x)))
        x = self.dropout1(self.act1(self.hidden1(x)))
        x = self.dropout2(self.act2(self.hidden2(x)))
        x = self.output(x)

        return x
    
class georgia_1(nn.Module):
    def __init__(self, config, win_past=20, features=19):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Flatten())

        for idx in range(len(config['neurons'])):
            if config['activations'][idx] == 'relu':
                activation = nn.ReLU()
            elif config['activations'][idx] == 'selu':
                activation = nn.SELU()
            elif config['activations'][idx] == 'sigmoid':
                activation = nn.Sigmoid()
            elif config['activations'][idx] == 'none':
                activation = "none"
            else:
                raise ValueError(f"Unrecognized activation function at index {idx}: {activations[idx]}")

            if idx == 0:
                self.layers.append(nn.Linear(features*win_past, config['neurons'][idx]))
            else:
                self.layers.append(nn.Linear(config['neurons'][idx - 1], config['neurons'][idx]))

            if activation != 'none':
                self.layers.append(activation)

            self.layers.append(nn.Dropout(config['dropouts'][idx]))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        return x


In [5]:
import random

def random_model_config():
    num_layers = random.randint(1, 10)  # Random number of layers between 1 and 10
    neurons = [random.randint(4, 512) for _ in range(num_layers - 1)] + [1]  # Last neuron always 1
    activations = [random.choice(['relu', 'selu', 'sigmoid', 'none']) for _ in range(num_layers)]
    dropouts = [round(random.uniform(0.0, 0.5), 2) for _ in range(num_layers - 1)] + [0.0]  # Last dropout is 0

    return {
        "neurons": neurons,
        "activations": activations,
        "dropouts": dropouts
    }

In [6]:
import time
import torch
from torch import optim
import torch_ops as ops
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x, y = data["x"], data["y"]
v_split = 0.7
v_cutoff = int(len(x)*v_split)
x_t = torch.tensor(x[:v_cutoff], dtype=torch.float32).to(device)
x_v = torch.tensor(x[v_cutoff:], dtype=torch.float32).to(device)
y_t = torch.tensor(y[:v_cutoff], dtype=torch.float32).to(device)
y_v = torch.tensor(y[v_cutoff:], dtype=torch.float32).to(device)

win_past, features = x.shape[1], x.shape[2]
epochs, batch_size, lr, l2_decay = 250, 32, 1e-5, 2e-5
best_score = 1
scores = []
for iteration in range(5000):
    model_config = random_model_config()
    model = georgia_1(model_config, win_past=win_past, features=features).to(device)
    loss_fn = nn.L1Loss()
    optimizer = optim.Adam(params=model.parameters(), lr=lr, weight_decay=l2_decay)

    # loss = ops.test_loop(x_t, y_t, model, loss_fn, batch_size)
    # test_loss = ops.test_loop(x_v, y_v, model, loss_fn, batch_size)
    # print("---------- Epoch 0 ----------")
    # print(f"loss: {loss:1.4f}, test loss: {test_loss:1.4f}")

    best_test_loss = 999
    t0 = time.perf_counter()
    for epoch in range(epochs):
        loss = ops.train_loop(x_t, y_t, model, loss_fn, optimizer, batch_size)
        test_loss_new = ops.test_loop(x_v, y_v, model, loss_fn, batch_size)
        best_test_loss = min(best_test_loss, test_loss_new)


        # if test_loss_new < test_loss or epoch == epochs - 1:
        #     print(f"---------- Epoch {epoch + 1} ----------")
        #     print(f"loss: {loss:1.4f}, test loss: {test_loss_new:1.4f}")
        
        # improvement = test_loss_new < test_loss
        # if improvement:
        #     test_loss = test_loss_new
        #     print(f"Checkpoint: test loss = {test_loss:1.4f} <---------------")
        #     torch.save(model.state_dict(), r"models/hanzo_0")

    t1 = time.perf_counter()
    model_config['score'] = best_test_loss
    scores.append(model_config)
    best_score = min(best_score, best_test_loss)

    print(f"Iteration {iteration}, t = {t1 - t0:1.2f}, score = {best_test_loss:1.4f}, best score = {best_score:1.4f}")

Iteration 0, t = 4.05, score = 0.1727, best score = 0.1727
Iteration 1, t = 4.97, score = 0.1709, best score = 0.1709


KeyboardInterrupt: 

In [44]:
model.parameters()

<generator object Module.parameters at 0x000002187F7B82E0>